<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object
import requests

# Define the URL of the Falcon9 Launch Wiki page
url = 'https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922'

# Perform an HTTP GET request to fetch the page
response = requests.get(url)

# Check the status of the response
if response.status_code == 200:
    print("Request was successful!")
    html_content = response.text  # Store the HTML content
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

Request was successful!


Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
from bs4 import BeautifulSoup

# Assuming 'response' contains the HTML response from the GET request
html_content = response.text

# Create a BeautifulSoup object and specify the parser
soup = BeautifulSoup(html_content, 'html.parser')

# Print the title of the page to verify
print(soup.title.string)


List of Falcon 9 and Falcon Heavy launches - Wikipedia


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
# Use soup.title attribute
from bs4 import BeautifulSoup
import requests

# Define the URL of the Falcon9 Launch Wiki page
url = 'https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922'

# Perform an HTTP GET request to fetch the page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful!")
    html_content = response.text  # Store the HTML content

    # Create a BeautifulSoup object and specify the parser
    soup = BeautifulSoup(html_content, 'html.parser')

    # Print the title of the page to verify
    print("Page Title:", soup.title.string)
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

Request was successful!
Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
from bs4 import BeautifulSoup
import requests

# Define the URL of the Falcon9 Launch Wiki page
url = 'https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922'

# Perform an HTTP GET request to fetch the page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful!")
    html_content = response.text  # Store the HTML content

    # Create a BeautifulSoup object and specify the parser
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all tables on the page
    tables = soup.find_all('table')

    # Loop through each table and extract header names
    for idx, table in enumerate(tables):
        print(f"\nTable {idx + 1} headers:")
        
        # Find all table headers within the table
        headers = table.find_all('th')
        
        # Extract and print the text of each header
        for header in headers:
            print(header.text.strip())
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Request was successful!

Table 1 headers:

Table 2 headers:

Table 3 headers:
Flight No.
Date andtime (UTC)
Version,Booster [b]
Launch site
Payload[c]
Payload mass
Orbit
Customer
Launchoutcome
Boosterlanding
1
2
3
4
5
6
7

Table 4 headers:
Flight No.
Date andtime (UTC)
Version,Booster[b]
Launch site
Payload[c]
Payload mass
Orbit
Customer
Launchoutcome
Boosterlanding
8
9
10
11
12
13

Table 5 headers:
Flight No.
Date andtime (UTC)
Version,Booster[b]
Launch site
Payload[c]
Payload mass
Orbit
Customer
Launchoutcome
Boosterlanding
14
15
16
17
18
19
20

Table 6 headers:
Flight No.
Date andtime (UTC)
Version,Booster[b]
Launch site
Payload[c]
Payload mass
Orbit
Customer
Launchoutcome
Boosterlanding
21
22
23
24
25
26
27
28
N/A [e]

Table 7 headers:
Flight No.
Date andtime (UTC)
Version,Booster[b]
Launch site
Payload[c]
Payload mass
Orbit
Customer
Launchoutcome
Boosterlanding
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46

Table 8 headers:
Flight No.
Date andtime (UTC)
Version,Booster[b]


Starting from the third table is our target table contains the actual launch records.


In [10]:
# Let's print the third table and check its content
from bs4 import BeautifulSoup
import requests

# Define the URL of the Falcon9 Launch Wiki page
url = 'https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922'

# Perform an HTTP GET request to fetch the page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful!")
    html_content = response.text  # Store the HTML content

    # Create a BeautifulSoup object and specify the parser
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all tables on the page
    tables = soup.find_all('table')

    # Check if there are enough tables on the page
    if len(tables) >= 3:
        # The third table is at index 2 (since indices start at 0)
        target_table = tables[2]
        print("\nExtracting headers from the third table:")

        # Find all table headers within the target table
        headers = target_table.find_all('th')

        # Extract and print the text of each header
        for header in headers:
            print(header.text.strip())
    else:
        print("There are not enough tables on the page.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
first_launch_table = html_tables[2]
print(first_launch_table)

Request was successful!

Extracting headers from the third table:
Flight No.
Date andtime (UTC)
Version,Booster [b]
Launch site
Payload[c]
Payload mass
Orbit
Customer
Launchoutcome
Boosterlanding
1
2
3
4
5
6
7


NameError: name 'html_tables' is not defined

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [11]:
column_names = []
from bs4 import BeautifulSoup
import requests

# Define the URL of the Falcon9 Launch Wiki page
url = 'https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922'

# Function to extract column name from a header element
def extract_column_from_header(th_element):
    return th_element.get_text(separator=" ").strip()

# Perform an HTTP GET request to fetch the page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful!")
    html_content = response.text  # Store the HTML content

    # Create a BeautifulSoup object and specify the parser
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all tables on the page
    tables = soup.find_all('table')

    # Check if there are enough tables on the page (at least 3)
    if len(tables) >= 3:
        # The third table is at index 2 (since indices start at 0)
        target_table = tables[2]
        print("\nExtracting headers from the third table:")

        # Find all table headers within the target table
        headers = target_table.find_all('th')

        # Extract and print the column names
        for header in headers:
            column_name = extract_column_from_header(header)
            print(column_name)
    else:
        print("There are not enough tables on the page.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Request was successful!

Extracting headers from the third table:
Flight No.
Date and time ( UTC )
Version, Booster   [ b ]
Launch site
Payload [ c ]
Payload mass
Orbit
Customer
Launch outcome
Booster landing
1
2
3
4
5
6
7


Check the extracted column names


In [12]:
print(column_names)

[]


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [14]:
launch_dict= dict.fromkeys(column_names)

import pandas as pd
from bs4 import BeautifulSoup
import requests

# Define the URL of the Falcon9 Launch Wiki page
url = 'https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922'

# Function to extract column name from a header element
def extract_column_from_header(th_element):
    return th_element.get_text(separator=" ").strip()

# Perform an HTTP GET request to fetch the page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful!")
    html_content = response.text  # Store the HTML content

    # Create a BeautifulSoup object and specify the parser
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all tables on the page
    tables = soup.find_all('table')

    # Check if there are enough tables on the page (at least 3)
    if len(tables) >= 3:
        # The third table is at index 2 (since indices start at 0)
        target_table = tables[2]
        print("\nExtracting headers from the third table:")

        # Extract column names
        headers = target_table.find_all('th')
        column_names = [extract_column_from_header(header) for header in headers]

        # Create an empty dictionary to hold the data
        launch_data = {col: [] for col in column_names}

        # Extract the table rows
        rows = target_table.find_all('tr')[1:]  # Skip the header row

        for row in rows:
            # Extract each cell (td) from the row
            cells = row.find_all('td')

            # Check if the row has the correct number of cells (to match columns)
            if len(cells) == len(column_names):
                for idx, cell in enumerate(cells):
                    # Append the cell data to the corresponding column in the dictionary
                    launch_data[column_names[idx]].append(cell.get_text(separator=" ").strip())

        # Convert the dictionary into a Pandas DataFrame
        df = pd.DataFrame(launch_data)

        # Display the DataFrame
        print(df.head())
    else:
        print("There are not enough tables on the page.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

Request was successful!

Extracting headers from the third table:
Empty DataFrame
Columns: [Flight No., Date and time ( UTC ), Version, Booster   [ b ], Launch site, Payload [ c ], Payload mass, Orbit, Customer, Launch outcome, Booster landing, 1, 2, 3, 4, 5, 6, 7]
Index: []


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [16]:
extracted_row = 0
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Define the URL of the Falcon9 Launch Wiki page
url = 'https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922'

# Function to extract column name from a header element
def extract_column_from_header(th_element):
    return th_element.get_text(separator=" ").strip()

# Function to clean cell content by removing unwanted reference links and extra spaces
def clean_cell_data(cell):
    # Remove reference links and unnecessary characters
    cleaned_data = cell.get_text(separator=" ").strip()
    return cleaned_data.replace("\xa0", " ").replace("[edit]", "")  # Clean up non-breaking spaces and other annotations

# Perform an HTTP GET request to fetch the page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful!")
    html_content = response.text  # Store the HTML content

    # Create a BeautifulSoup object and specify the parser
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all tables on the page
    tables = soup.find_all('table')

    # Check if there are enough tables on the page (at least 3)
    if len(tables) >= 3:
        # The third table is at index 2 (since indices start at 0)
        target_table = tables[2]
        print("\nExtracting headers from the third table:")

        # Extract column names
        headers = target_table.find_all('th')
        column_names = [extract_column_from_header(header) for header in headers]

        # Create an empty dictionary to hold the data
        launch_dict = {col: [] for col in column_names}

        # Extract the table rows
        rows = target_table.find_all('tr')[1:]  # Skip the header row

        for row in rows:
            # Extract each cell (td) from the row
            cells = row.find_all('td')

            # Check if the row has the correct number of cells (to match columns)
            if len(cells) == len(column_names):
                for idx, cell in enumerate(cells):
                    # Clean and append the cell data to the corresponding column in the dictionary
                    launch_dict[column_names[idx]].append(clean_cell_data(cell))
            else:
                # Handle cases with inconsistent number of cells (e.g., missing data or annotations)
                print(f"Row length mismatch. Expected {len(column_names)} but got {len(cells)}. Filling with N/A.")
                for idx in range(len(column_names)):
                    if idx < len(cells):
                        launch_dict[column_names[idx]].append(clean_cell_data(cells[idx]))
                    else:
                        launch_dict[column_names[idx]].append("N/A")  # Fill missing columns with N/A

        # Convert the dictionary into a Pandas DataFrame
        df = pd.DataFrame(launch_dict)

        # Display the DataFrame
        print(df.head())
    else:
        print("There are not enough tables on the page.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Request was successful!

Extracting headers from the third table:
Row length mismatch. Expected 17 but got 9. Filling with N/A.
Row length mismatch. Expected 17 but got 1. Filling with N/A.
Row length mismatch. Expected 17 but got 9. Filling with N/A.
Row length mismatch. Expected 17 but got 1. Filling with N/A.
Row length mismatch. Expected 17 but got 9. Filling with N/A.
Row length mismatch. Expected 17 but got 1. Filling with N/A.
Row length mismatch. Expected 17 but got 9. Filling with N/A.
Row length mismatch. Expected 17 but got 5. Filling with N/A.
Row length mismatch. Expected 17 but got 1. Filling with N/A.
Row length mismatch. Expected 17 but got 9. Filling with N/A.
Row length mismatch. Expected 17 but got 1. Filling with N/A.
Row length mismatch. Expected 17 but got 9. Filling with N/A.
Row length mismatch. Expected 17 but got 1. Filling with N/A.
Row length mismatch. Expected 17 but got 9. Filling with N/A.
Row length mismatch. Expected 17 but got 1. Filling with N/A.
    

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [17]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
